## Prepare data reader for a given chain and date

In [ ]:
from op_analytics.coreutils.duckdb_inmem import init_client
from op_analytics.coreutils.partitioned import DataReader, DataLocation
from op_analytics.datapipeline.etl.intermediate.construct import construct_data_readers

from op_analytics.datapipeline.models.compute.udfs import create_duckdb_macros


# Define the input data range.
read_batches: list[DataReader] = construct_data_readers(
    chains=["op"],
    range_spec="@20241030:+1",
    read_from=DataLocation.GCS
)


# Select input for one date and build the intermediate model inputs.
batch = read_batches[0]


duckdb_client = init_client()
create_duckdb_macros(duckdb_client)


2024-11-19 13:41:35 [info     ] prepared 1 input batches.      filename=reader.py lineno=139


## Run the model

This automatically registers the model outputs as duckdb tables.

In [ ]:
from op_analytics.datapipeline.models.compute.registry import REGISTERED_INTERMEDIATE_MODELS, load_model_definitions
from op_analytics.datapipeline.models.compute.testutils import execute_model_in_memory

load_model_definitions()
execute_model_in_memory(
    duckdb_client=duckdb_client,
    model=REGISTERED_INTERMEDIATE_MODELS["daily_address_summary"],
    data_reader=batch
)

# The duckdb database will have the following:
#   - input tables
#   - views used by the model
#   - model outputs
# 
# You can use duckdb to inspect any of the above results.
duckdb_client.sql("SHOW TABLES")

2024-11-19 13:41:35 [info     ] Executing model...             filename=testutils.py lineno=201
dict_keys(['summary_v1'])


┌───────────────────────┐
│         name          │
│        varchar        │
├───────────────────────┤
│ blocks                │
│ daily_address_summary │
│ summary_v1            │
│ transaction_fees      │
│ transactions          │
└───────────────────────┘

## Verify model results

In [6]:
duckdb_client.sql("SELECT * FROM summary_v1 LIMIT 10")

┌────────────┬─────────┬──────────┬────────────────────────────────────────────┬────────┬────────────────┬───────────┬───────────────────┬──────────────────┬──────────────────┬────────────────────┬───────────┬───────────┬────────────────────┬─────────────────────┬─────────────────────┬───────────────────┬───────────────────┬─────────────────┬─────────────────────────┬────────────────┬─────────────────┬─────────────────────────┬─────────────────┬─────────────────────────┬───────────────────────┬────────────────────────┬───────────────────────┬───────────────────────┬─────────────────────────┬────────────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬───────────────────────┬────────────────────────┬────────────────────────────┬────────────────────────┬──────────────────────┐
│     dt     │  chain  │ chain_id │                  address                   │ tx_cnt │ success_tx_cnt │ block_cnt │ success_block_cnt │ block_number_min │ block_number_max

### You can also convert the results to dataframes to inspect them in more familiar ways

In [7]:
duckdb_client.sql("SELECT * FROM summary_v1 LIMIT 10").pl().head()

dt,chain,chain_id,address,tx_cnt,success_tx_cnt,block_cnt,success_block_cnt,block_number_min,block_number_max,active_block_range,nonce_min,nonce_max,active_nonce_range,block_timestamp_min,block_timestamp_max,active_time_range,active_hours_ucnt,to_address_ucnt,success_to_address_ucnt,method_id_ucnt,l2_gas_used_sum,success_l2_gas_used_sum,l1_gas_used_sum,success_l1_gas_used_sum,tx_fee_sum_eth,success_tx_fee_sum_eth,l2_fee_sum_eth,l2_base_fee_sum_eth,l2_priority_fee_sum_eth,l2_base_legacy_fee_sum_eth,l1_fee_sum_eth,l1_base_fee_sum_eth,l1_blob_fee_sum_eth,l2_gas_price_avg_gwei,l2_base_price_avg_gwei,l2_priority_price_avg_gwei,l1_base_price_avg_gwei,l1_blob_fee_avg_gwei
str,str,i32,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,u32,u32,u32,i64,i64,i64,i64,"decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,0]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,19]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]","decimal[38,10]"
"""2024-10-30""","""op""",10,"""0xc96faba4ff9941a55754d7a4b02a…",32838,32838,32206,32206,127323812,127367011,43200,1376386,1409223,32838,1730246401,1730332799,86398,24,2,2,2,14534607387,14534607387,52540800,52540800,0.0066643696969443920,0.0066643696969443920,0.0005774185478563580,0.0000070996380281920,0.0005702894569369060,0.0000000988002535930,0.0060869511490880340,0.0033354166620712290,0.0027515344870333490,0.0000397270,0.0000004880,0.0000392370,12.1450949000,0.8261693690
"""2024-10-30""","""op""",10,"""0xbfec073d98defc97d1e66a141b35…",7256,7256,7129,7129,127323814,127367010,43197,479774,487029,7256,1730246405,1730332797,86392,24,6,6,4,4303252798,4303252798,11613388,11613388,0.0016303929773812910,0.0016303929773812910,0.0000140958518230760,0.0000020090494646810,0.0000120868023583950,0.0000000000000000000,0.0016162971255582150,0.0007437920124578070,0.0008725049283754900,0.0000032760,0.0000004670,0.0000028090,12.2529334340,1.1852221590
"""2024-10-30""","""op""",10,"""0x3d005d08da1e75f0175fc8e1d654…",2,2,2,2,127334003,127334146,144,245,246,2,1730266783,1730267069,286,1,1,1,1,211640,211640,3200,3200,0.0000002213605725760,0.0000002213605725760,0.0000000212501943280,0.0000000000861943280,0.0000000211640000000,0.0000000000000000000,0.0000002001103782480,0.0000001926751116490,0.0000000074352665990,0.0001004070,0.0000004070,0.0001000000,11.5192218080,0.0366553500
"""2024-10-30""","""op""",10,"""0x32828a5d49af803cdeebeca2c3bd…",6,6,6,6,127328590,127358821,30232,29484,29489,6,1730255957,1730316419,60462,6,1,1,1,648507,648507,18137,18137,0.0000027169155954940,0.0000027169155954940,0.0000006488076231390,0.0000000003005862700,0.0000000000000000000,0.0000027166150092240,0.0000020681079723550,0.0000010792459148110,0.0000009882017486690,0.0010004640,0.0000004640,0.0000000000,11.3841985170,0.8595496640
"""2024-10-30""","""op""",10,"""0xf72e10ba10c78b845e63328f83ae…",28,28,27,27,127333983,127366606,32624,35,62,28,1730266743,1730331989,65246,5,10,10,9,6544189,6544189,148529,148529,0.0000212599598362850,0.0000212599598362850,0.0000061559022846970,0.0000000030152846970,0.0000061528870000000,0.0000000000000000000,0.0000151040575515880,0.0000086545590175720,0.0000064482654892970,0.0009406670,0.0000004610,0.0009402060,11.1475951170,0.6848925820
